In [1]:
import datetime as dt
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
month, day, year = 11, 5, 2019
url = 'https://www.sports-reference.com/cbb/boxscores/index.cgi?month='+str(month)+'&day='+str(day)+'&year='+str(year)
#url = 'https://www.sports-reference.com/cbb/boxscores/index.cgi?month=11&day=5&year=2019'
page = requests.get(url)
soup = BeautifulSoup(page.content, 'html.parser')


In [3]:
url = 'https://www.sports-reference.com/cbb/boxscores/index.cgi?month=1&day=20&year=2020'
page = requests.get(url)
soup = BeautifulSoup(page.content, 'html.parser')


In [4]:
url = 'https://www.sports-reference.com/cbb/boxscores/index.cgi?month=12&day=5&year=2019'
page = requests.get(url)
soup = BeautifulSoup(page.content, 'html.parser')

In [5]:
soup.find('td').find('a').text

'Furman'

In [6]:
scores = soup.find_all('td')
#for score in scores:
#    print(score)


In [7]:
i=0
for score in scores:
    if i%6 in [0,3]:
        print(score.find('a').text)
    if i%6 in [1,4]:
        print(int(score.text))
    i+=1
    if i == 6:
        break

Furman
78
Auburn
81


In [8]:
visitor_neutral = []
home_neutral = []
v_score = []
h_score =[]
dates = []
date =str(month)+'/'+str(day)+'/'+str(year)
i=0
for info in scores:
    if i%6 == 0:
        visitor_neutral.append(info.find('a').text)
    elif i%6 == 3:
        home_neutral.append(info.find('a').text)
    elif i%6 == 1:
        v_score.append(int(info.text))
    elif i%6 == 4:
        h_score.append(int(info.text))
    else:
        dates.append(date)
    i+=1

In [9]:
print(visitor_neutral[0], v_score[0], home_neutral[0], h_score[0])

Furman 78 Auburn 81


In [10]:
df = pd.DataFrame(zip(dates, visitor_neutral, v_score, home_neutral, h_score), 
               columns =['Date','visitor_neutral', 'PTS','home_neutral','PTS']) 
df['Date']= pd.to_datetime(df['Date'])
df.head()

,Date,visitor_neutral,PTS,home_neutral,PTS
0,2019-11-05,Furman,78,Auburn,81
1,2019-11-05,Central Arkansas,69,Wichita State,95
2,2019-11-05,Elon,66,High Point,70
3,2019-11-05,Howard,94,Hampton,91
4,2019-11-05,Louisiana Tech,74,Mississippi State,67


In [132]:
month, day, year = 11, 25, 2020

current_date = dt.date(year,month,day)
visitor_neutral = []
home_neutral = []
v_score = []
h_score =[]
dates = []
master_df = pd.DataFrame()
date =str(month)+'/'+str(day)+'/'+str(year)
while(current_date != dt.date(2021, 3, 22)):
#reset lists
    visitor_neutral = []
    home_neutral = []
    v_score = []
    h_score =[]
    dates = []
    date =str(month)+'/'+str(day)+'/'+str(year)
#Get score page for date
    url = 'https://www.sports-reference.com/cbb/boxscores/index.cgi?month='+str(month)+'&day='+str(day)+'&year='+str(year)
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    scores = soup.find_all('td')
#Check for games that day
    content = soup.find('div', {"class":"section_content"})
    if (content.text.strip() == "No games found."):
# increment day, month, year
        current_date += dt.timedelta(days=1)
        day = current_date.day
        month = current_date.month
        year = current_date.year        
        continue
#get lists of all games / scores
    i=0
    for info in scores:
        if ("colspan" not in info.attrs):
            if i%6 == 0:
                visitor_neutral.append(info.find('a').text)
            elif i%6 == 3:
                home_neutral.append(info.find('a').text)
            elif i%6 == 1:
                if not info.text:
                    v_score.append(0)
                else:
                    v_score.append(int(info.text))
            elif i%6 == 4:
                if not info.text:
                    h_score.append(0)
                else:
                    h_score.append(int(info.text))
            else:
                dates.append(date)
            i+=1

        
        
        
        
        
#make dataframe of lists
    df = pd.DataFrame(zip(dates, visitor_neutral, v_score, home_neutral, h_score), 
               columns =['Date','visitor_neutral', 'PTS','home_neutral','PTS']) 
    df['Date']= pd.to_datetime(df['Date'])
#combine with master dataframe
    master_df = pd.concat([master_df, df])
# increment day, month, year
    current_date += dt.timedelta(days=1)
    day = current_date.day
    month = current_date.month
    year = current_date.year

    

    

In [133]:
master_df.tail()

,Date,visitor_neutral,PTS,home_neutral,PTS
9,2021-03-20,VCU,0,Oregon,0
10,2021-03-20,Missouri,0,Oklahoma,0
11,2021-03-20,St. Bonaventure,0,LSU,0
12,2021-03-20,Dayton,0,Memphis,0
13,2021-03-20,Mississippi State,0,Saint Louis,0


In [134]:
master_df.to_csv("2020_2021_Schedule_updated.csv")

In [135]:
master_df[(master_df["home_neutral"]=="Michigan")]

,Date,visitor_neutral,PTS,home_neutral,PTS
16,2020-11-25,Bowling Green State,82,Michigan,96
7,2020-11-29,Oakland,71,Michigan,81
9,2020-12-02,Ball State,65,Michigan,84
12,2020-12-06,UCF,58,Michigan,80
49,2020-12-09,Toledo,71,Michigan,91
23,2020-12-13,Penn State,58,Michigan,62
1,2021-01-03,Northwestern,66,Michigan,85
3,2021-01-06,Minnesota,57,Michigan,82
1,2021-01-12,Wisconsin,54,Michigan,77
2,2021-01-19,Maryland,63,Michigan,87
